In [182]:
%load_ext autoreload
%autoreload 2

import time
import itertools

import numpy as np
import pickle as pkl

from scipy import spatial as spatial
from scipy import sparse as sp
from sklearn.preprocessing import scale

from steenroder import *
import gudhi

from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
def sphere(n=100, r=1, noise=None):
    '''(r, u, v) \mapsto (r sin u cos v, y = r sin u sin v, z = r cos u)'''
    u = np.random.random((n,)) * 2.0 * np.pi
    v = np.random.random((n,)) * np.pi
    rad = np.ones((n,)) * r

    data = np.zeros((n, 3))

    data[:, 0] = rad * np.cos(u) * np.cos(v)
    data[:, 1] = rad * np.cos(u) * np.sin(v)
    data[:, 2] = rad * np.sin(u)

    if noise: 
        data += noise * np.random.randn(*data.shape)

    return data

def real_proj(S):
    """(x,y,z) \mapsto (xy, xz, y^2−z^2, 2yz)"""
    RP = np.empty((n, 4))

    RP[:,0] = S[:,0]*S[:,1]
    RP[:,1] = S[:,0]*S[:,2]
    RP[:,2] = S[:,1]*S[:,1] - S[:,2]*S[:,2]
    RP[:,3] = 2*S[:,1]*S[:,2]

    return RP

In [184]:
n, r, noise = 115, 1, 0
s2 = sphere(n, r, noise)
rp2 = real_proj(s2)

rips_complex_rp2 = gudhi.RipsComplex(points=rp2)
simplex_tree_rp2 = rips_complex_rp2.create_simplex_tree(max_dimension=1)  # Only get the 1-skeleton this time

In [185]:
simplex_tree_rp2.collapse_edges(nb_iterations=10)  # Run this cell several times if you want to iteratively collapse

filtration_rp2 = []
for s in simplex_tree_rp2.get_filtration():
    filtration_rp2.append(s)

print(f"Filtration with {len(filtration_rp2)} simplices")

Filtration with 1267 simplices


In [186]:
simplex_tree_rp2.expansion(3)  # Get the three-simplices after collapse

filtration_rp2 = []
for s in simplex_tree_rp2.get_filtration():
    filtration_rp2.append(s)

print(f"Filtration with {len(filtration_rp2)} simplices")

Filtration with 20967 simplices


In [187]:
barcode = simplex_tree_rp2.persistence(homology_coeff_field=2)
filtered_barcode = [bar for bar in barcode if bar[1][1]-bar[1][0] > r*0.5]
filtered_barcode

[(0, (0.0, inf)), (0, (0.0, 0.5416194778902602))]

In [188]:
import time

t0 = time.time()
barcode_rp2, st_barcode_rp2 = barcodes(1, [tuple(s[0]) for s in filtration_rp2])
t1 = time.time()

print('time:', t1-t0)
print(st_barcode_rp2)

time: 5.902426719665527
[[], [], [], [(20255, 20270)]]
